In [18]:
import numpy as np
import pandas as pd
import io
import math
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [19]:
from google.colab import files
data = pd.read_csv('/content/drive/MyDrive/sem6 myfolder/mldatasets/BuyComputer.csv')
data.drop(columns=['User ID',],axis=1,inplace=True)
data.head()
x = data[["Age", "EstimatedSalary"]]
y = data["Purchased"]
x, y

(     Age  EstimatedSalary
 0     19            19000
 1     35            20000
 2     26            43000
 3     27            57000
 4     19            76000
 ..   ...              ...
 395   46            41000
 396   51            23000
 397   50            20000
 398   36            33000
 399   49            36000
 
 [400 rows x 2 columns], 0      0
 1      0
 2      0
 3      0
 4      0
       ..
 395    1
 396    1
 397    1
 398    0
 399    1
 Name: Purchased, Length: 400, dtype: int64)

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(x,y, train_size=0.75, random_state=88)
X_train.shape, X_test.shape, y_train.shape, y_test.shape, X_train.head()
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
y_train, y_test = y_train.tolist(), y_test.tolist()
X_test = sc.transform(X_test)
X_train[:5], y_train[:5]

(array([[ 0.99513555, -0.82080737],
        [ 0.99513555, -1.14326741],
        [ 1.474796  ,  0.08794365],
        [-0.25198162,  0.17588729],
        [ 0.13174674,  0.1172582 ]]), [1, 1, 1, 0, 0])

In [21]:
def model(x_age, x_salary, theta_0, theta_age, theta_salary):
  return 1 / (1 + math.exp(-1 * (theta_0 + theta_age*x_age + theta_salary*x_salary)))
math.exp(2), math.exp(1)**2, np.exp(1)
model(X_train[1][0], X_train[1][1],1,1,1), y[1]

(0.7009588808924954, 0)

In [22]:
def loss(y_pred, y_actual):
  return (y_actual) and (-1 * math.log10(y_pred)) or (-1 * math.log10(1 - y_pred))
loss(model(X_train[0][0], X_train[0][1],1,1,1), y_train[0]), y_train[0], model(X_train[0][0], X_train[0][1],1,1,1)
def update(theta, lr, m, y_pred, y_actual, feature_i=0):
  summation = 0
  for i in range(len(y_pred)):
    summation = summation + (y_pred[i] - y_actual[i])*(feature_i and feature_i[i] or 1)
  return theta - (lr/m)*summation

In [23]:
theta_0, theta_age, theta_salary = 1,1,1
epochs = 10 # 100
learning_rate = 10
X_train_df = pd.DataFrame(X_train, columns=["Age", "EstimatedSalary"])
age = X_train_df["Age"].tolist()
estimated_salary = X_train_df["EstimatedSalary"].tolist()
for j in range(epochs):
  y_train_pred = []
  m = len(X_train)
  for i in range(m):
    y_train_pred.append(model(X_train[i][0], X_train[i][1], theta_0, theta_age, theta_salary))
    loss_value = loss(y_train_pred[i], y_train[i])
  theta_0 = update(theta_0, learning_rate, m, y_train_pred, y_train)
  theta_age = update(theta_age, learning_rate, m, y_train_pred, y_train, age)
  theta_salary = update(theta_salary, learning_rate, m, y_train_pred, y_train, estimated_salary)

y_test_pred = []
for i in range(len(X_test)):
  if model(X_test[i][0], X_test[i][1], theta_0, theta_age, theta_salary) < 0.5:
    y_test_pred.append(0)
  else:
    y_test_pred.append(1)
TP, FP, TN, FN = 0,0,0,0

for i in range(len(X_test)):
  if y_test_pred[i]==1 and y_test[i]==1:
    TP = TP+1
  if y_test[i]==0 and y_test_pred[i]==1:
    FP = FP+1
  if y_test_pred[i]==0 and y_test[i]==0:
    TN = TN+1
  if y_test_pred[i]==0 and y_test[i]==1:
    FN = FN+1
P = TP/(TP+FP)
A = (TP+TN)/len(X_test)
R = TP/(TP+FN)
F1_Score = (2*P*R)/(P+R)
print(f"A: {A}\nP: {P}\nR: {R}\nF Score: {F1_Score}")

 

A: 0.85
P: 0.8787878787878788
R: 0.725
F Score: 0.7945205479452054


In [24]:
model=LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [25]:
y_pred = model.predict(X_test)
print(y_pred)
print("A ",metrics.accuracy_score(y_test, y_pred))
print("P ",metrics.precision_score(y_test,y_pred))
print("R ",metrics.recall_score(y_test,y_pred))
print("F1 ",metrics.f1_score(y_test,y_pred))

[0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0
 1 1 0 1 0 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 0 1
 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0]
A  0.84
P  0.875
R  0.7
F1  0.7777777777777777
